# Setting

In [1]:
# ## (1) TabPFN을 Internet off에서 사용하기 위한, 패키지 다운로드
# ##     -> Internet on(또는 Local)에서 아래의 코드로 실행 후, pip-packages.zip을 다운로드

# ## (1-1) /kaggle/working(Local에서는 해당 Path)에 /pip-packages/에 tabpfn.whl파일 다운로드
# !pip download tabpfn --no-deps -d ./pip-packages

# ## (1-2) tabpfn install
# !pip install tabpfn

# ## (1-3) 모델실행하면 .cpkt파일 생성됨 (cpu, cuda:0에 따라서 설치되는파일이 다른 것으로 보이므로 선택잘해야함)
# from tabpfn import TabPFNClassifier
# TabPFNClassifier(N_ensemble_configurations=64,device='cpu')

# ## (1-4) .cpkt파일을 /pip-packages/로 이동
# !mv /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt pip-packages/

# ## (1-5) /pip-packages/에 필요한 파일들을 다 모았으므로, 압축해줌
# !zip -r pip-packages.zip pip-packages

# ## (1-6) pip-packages.zip을 다운로드

# ## (1-7) Kaggle Dataset에 업로드
#        -> Kaggle Dataset 또는 현재창 우측의 +Add Data 옆의 데이터 업로드를 누른 후,
#        -> (1)에서 다운받은 pip-packages.zip을 pip-packages-icr의 이름으로 업로드
#        -> 업로드 된 zip파일은 나의 kaggle dataset에 저장되므로 영구적으로 사용가능

In [2]:
## (2) TabPFN을 Internet off에서 사용하기 위한, 패키지 업로드

## (2-1) Kaggle Dataset 로드
#        -> 우측의 +Add Data > Your Datasets > pip-packages-icr 클릭

## (2-2) 아래의 쿼리로 Internet off 상태로 다운로드 가능함
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages/

## (2-3) TabPFN의 .cpkt파일을 불러오는 /opt/conda/lib으로 폴더생성 및 복사
##       -> 이후에 모델실행가능해짐
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages/
Processing /kaggle/input/pip-packages-icr/pip-packages/tabpfn-0.1.9-py3-none-any.whl


In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
class CFG:
    REF_PATH = '/kaggle/input/icr-identify-age-related-conditions/'
    #REF_PATH = './data/'
    
    VERBOSE = False
    
    #DEVICE = 'cuda:0'
    DEVICE = 'cpu'
        
    ADD_INTERACTION = False #False
    FS_ALPHA = 0.01
    
    SEED = 0
    N_SPLITS = 5
    EPOCHS = 50000
    ES = 300
    LR = 0.001

<br></br>

# Data Load

In [5]:
train_df  = pd.read_csv(CFG.REF_PATH+'train.csv')
test_df   = pd.read_csv(CFG.REF_PATH+'test.csv')
greeks_df = pd.read_csv(CFG.REF_PATH+'greeks.csv')

In [6]:
train_df.columns  = [col.replace(' ','') for col in train_df.columns]
test_df.columns   = [col.replace(' ','') for col in test_df.columns]
greeks_df.columns = [col.replace(' ','') for col in greeks_df.columns]

In [7]:
train_df.shape, greeks_df.shape

((617, 58), (617, 6))

In [8]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [9]:
greeks_df.head()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020


In [10]:
list(set(train_df.Id)-set(greeks_df.Id)), list(set(greeks_df.Id)-set(train_df.Id))

([], [])

In [11]:
# [train.csv] The training set
#
# - Id 
#   : Unique identifier for each observation.
#
# - AB-GL 
#   : Fifty-six anonymized health characteristics.
#   : All are numeric except for EJ, which is categorical.
#
# - Class
#   : A binary target
#   : 1 indicates the subject has been diagnosed with one of the three conditions, 
#   : 0 indicates they have not.
#
# [greeks.csv] Supplemental metadata, only available for the training set.
#
# - Alpha
#   : Identifies the type of age-related condition, if present.
#   - A : No age-related condition. Corresponds to class 0
#   - B, D, G : The three age-related conditions. Correspond to class 1
# - Beta, Gamma, Delta
#   : Three experimental characteristics.
# - Epsilon
#   : The date the data for this subject was collected.
#   : Note that all of the data in the test set was collected after the training set was collected.

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['Class'] = le.fit_transform(greeks_df['Alpha'])
assert le.classes_[0]=='A', \
    "First label must be 'A'"

In [13]:
from datetime import datetime
times = greeks_df.Epsilon.copy()
times[greeks_df.Epsilon != 'Unknown'] = greeks_df.Epsilon[greeks_df.Epsilon != 'Unknown']\
    .map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
times[greeks_df.Epsilon == 'Unknown'] = np.nan

train_df['epsilon'] = times
test_df ['epsilon'] = train_df.epsilon.max() + 1

In [14]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class,epsilon
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,737137.0
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,NaN
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,NaN
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,NaN
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,2,737509.0


<br></br>

# Preprocess

## Imputation

In [15]:
# define features type
target_feature = 'Class'
unuse_features = ['Id']
cat_features = ['EJ']
num_features = list(set(train_df.columns)-set([target_feature]+cat_features+unuse_features))

In [16]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
train_df[num_features] = imputer.fit_transform(train_df[num_features])
test_df [num_features] = imputer.transform(test_df[num_features])

<br>

## Add Interactions

In [17]:
import warnings
from scipy.stats import pearsonr

class AddInteraction:
    def __init__(self):
        warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
        
    def fit(self,data,features,corr_cutoff=0.7):
        self.fit_data = data
        self.features = features
        self.corr_cutoff = corr_cutoff
        self.interaction_features = []
        for i in range(len(self.features)):
            for j in range(len(self.features)):
                if i>j:
                    col_i = self.features[i]
                    col_j = self.features[j]
                    corr_i,_ = pearsonr(data[col_i],data[col_i]*data[col_j])
                    corr_j,_ = pearsonr(data[col_j],data[col_i]*data[col_j])
                    if (np.abs(corr_i)<corr_cutoff) & (np.abs(corr_j)<corr_cutoff):
                        self.interaction_features.append([col_i,col_j])
                    
    def transform(self,data):
        d = data.copy()
        for col_i,col_j in self.interaction_features:
            d[f'{col_i}*{col_j}'] = d[col_i]*d[col_j]
        return d

In [18]:
train_df2 = train_df.copy()
test_df2  = test_df.copy()

if CFG.ADD_INTERACTION:
    interaction_maker = AddInteraction()
    interaction_maker.fit(train_df2,num_features,corr_cutoff=0.7)
    train_df2 = interaction_maker.transform(train_df2)
    test_df2  = interaction_maker.transform(test_df2)

In [19]:
train_df2.shape,test_df2.shape

((617, 59), (5, 58))

In [20]:
# define numerical type features
num_features = list(set(train_df2.columns)-set([target_feature]+cat_features+unuse_features))

<br>

## Feature Selection

### Numerical

In [21]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [22]:
def log_offset(x):
    if min(x)>0:
        offset = 0
    elif min(x)==0:
        offset = 1e-3
    else:
        offset = min(x)+1e-3
        print('minimum = {:.3f}'.format(min(x)))
    return np.log(x+offset)

In [23]:
CFG.FS_ALPHA

0.01

In [24]:
pvalue_list = []
for col in tqdm(num_features):
    ols_data = train_df2[[col,target_feature]].copy()
    ols_data.columns = ['feature','target']

    formula = 'feature ~ C(target)'
    model = ols(formula=formula,data=ols_data).fit()
    pvalue = anova_lm(model)['PR(>F)']['C(target)']
    
    pvalue_list.append([col,pvalue])
    
pvalue_df = pd.DataFrame(pvalue_list,columns=['feature','pvalue']).sort_values('pvalue',ascending=False)
unsignificant_features = pvalue_df[pvalue_df.pvalue>CFG.FS_ALPHA].feature.tolist()

100%|██████████| 56/56 [00:00<00:00, 57.52it/s]


In [25]:
log_pvalue_list = []
for col in tqdm(unsignificant_features):
    ols_data = train_df2[[col,target_feature]].copy()
    ols_data.columns = ['feature','target']
    ols_data['feature'] = log_offset(ols_data['feature'])

    formula = 'feature ~ C(target)'
    model = ols(formula=formula,data=ols_data).fit()
    pvalue = anova_lm(model)['PR(>F)']['C(target)']
    
    log_pvalue_list.append([col,pvalue])
    
log_pvalue_df = pd.DataFrame(log_pvalue_list,columns=['feature','pvalue']).sort_values('pvalue',ascending=False)
delete_features = log_pvalue_df[log_pvalue_df.pvalue>CFG.FS_ALPHA].feature.tolist()

100%|██████████| 22/22 [00:00<00:00, 53.04it/s]


In [26]:
train_df3 = train_df2.copy()
test_df3  = test_df2.copy()

train_df3.drop(columns=delete_features,inplace=True)
test_df3 .drop(columns=delete_features,inplace=True)

log_features = list(set(unsignificant_features)-set(delete_features))
for col in log_features:
    train_df3[col] = log_offset(train_df3[col])
    test_df3 [col] = log_offset(test_df3 [col])

In [27]:
if CFG.VERBOSE:
    print('> Log Features:', log_features)
    print('> Delete Features:', delete_features)

In [28]:
train_df2.shape,train_df3.shape

((617, 59), (617, 42))

In [29]:
# define features type
target_feature = 'Class'
unuse_features = ['Id']
cat_features = ['EJ']
num_features = list(set(train_df3.columns)-set([target_feature]+cat_features+unuse_features))

<br></br>

# Modeling

- No Interaction, Binary
- No Interaction, MultiClass
- Interaction, Binary
- Interaction, MultiClass

In [30]:
from sklearn.model_selection import StratifiedKFold

In [31]:
def balanced_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    _balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    return _balanced_log_loss/(N_0+N_1)

In [32]:
class OneHotEncoder:
    def __init__(self):
        pass
    
    def fit(self,data,columns):
        self.transform_list = []
        for col in cat_features:
            for i,value in enumerate(sorted(data[col].unique())):
                if i>0:
                    self.transform_list.append([col,value])
        
    def transform(self,data):
        new_data = data.copy()
        for col,value in self.transform_list:
            new_data[f'{col}_{value}'] = np.where(new_data[col]==value,1,0)
        drop_columns = pd.unique(np.array(self.transform_list)[:,0])
        new_data.drop(columns=drop_columns,inplace=True)
        return new_data

In [33]:
def get_weighted_proba(ps):
    p = np.mean(ps,axis=0)
    class_0_est_instances = p[:,0].sum()
    others_est_instances = p[:,1:].sum()
    # we reweight the probs, since the loss is also balanced like this
    # our models out of the box optimize CE
    # with these changes they optimize balanced CE
    new_p = p * np.array([
        [1/(class_0_est_instances if i==0 else others_est_instances)
         for i in range(p.shape[1])]
    ])
    new_p = new_p / np.sum(new_p,axis=1,keepdims=1)
    new_p = np.concatenate((new_p[:,:1],np.sum(new_p[:,1:],1,keepdims=True)), 1)
    return new_p

def proba_multiclass_to_binary(ps,power=1):
    class_0_proba = ps[:,0].reshape(-1,1)**power
    class_1_proba = (1-class_0_proba).reshape(-1,1)
    return np.concatenate((class_0_proba,class_1_proba),axis=1)

In [34]:
from copy import deepcopy
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from tabpfn import TabPFNClassifier

In [35]:
# (1) CatBoost
params_catboost = {
    'random_state':CFG.SEED,
    'early_stopping_rounds' : CFG.ES,
    'learning_rate' : CFG.LR,
    'iterations' : CFG.EPOCHS,
    'loss_function': 'MultiClass',
    'auto_class_weights' : 'Balanced',
    'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
    'use_best_model' : True,
    'allow_writing_files' : False,
    'verbose' : False,
    'max_depth': 10,
    'l2_leaf_reg' : 1,
}
ref_catboost_model = CatBoostClassifier(**params_catboost)

# (2) XGBoost
params_xgboost = {
    'random_state':CFG.SEED,
    'early_stopping_rounds' : CFG.ES,
    'learning_rate' : CFG.LR,
    'n_estimators' : CFG.EPOCHS,
    'loss_function': 'MultiClass',
    'verbosity' : 0,
    'max_depth': 10,
    'n_jobs' : -1,
}
ref_xgboost_model = XGBClassifier(**params_xgboost)

# (3) TabPFN
ref_tabpfn_model = TabPFNClassifier(N_ensemble_configurations=64,seed=CFG.SEED,device=CFG.DEVICE)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [36]:
%%time
# 10분

proba_power = 1
skf = StratifiedKFold(n_splits=CFG.N_SPLITS,shuffle=True,random_state=CFG.SEED)

X = train_df3[cat_features+num_features]
y = train_df3[target_feature]

# one-hot encoding for XGB, TabPFN
oh_encoder = OneHotEncoder()
oh_encoder.fit(X,cat_features)
X_oh = oh_encoder.transform(X)

models = []
weights = []
preds = []

pbar = tqdm(skf.split(X,y),total=CFG.N_SPLITS)
for train_idx,val_idx in pbar:
    # define train/val dataset
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val  , y_val   = X.iloc[val_idx  ], y.iloc[val_idx  ]
    X_train_oh, X_val_oh = X_oh.iloc[train_idx], X_oh.iloc[val_idx]
    
    # dataset for CatBOost
    catboost_traindata = Pool(X_train,y_train,cat_features=cat_features)
    catboost_valdata   = Pool(X_val  ,y_val  ,cat_features=cat_features)
    
    # model copy
    model_catboost = deepcopy(ref_catboost_model)
    model_xgboost  = deepcopy(ref_xgboost_model)
    model_tabpfn   = deepcopy(ref_tabpfn_model)
    
    # modeling - catboost
    pbar.set_description('[1/3] CatBoost Fitting...')
    model_catboost.fit(catboost_traindata,eval_set=catboost_valdata)
    pred_catboost = model_catboost.predict_proba(X_val)
    p_catboost = proba_multiclass_to_binary(pred_catboost,power=proba_power)
    loss_catboost = balanced_log_loss(np.where(y_val==0,0,1), p_catboost[:, 1])
    
    # modeling - xgboost
    pbar.set_description('[2/3] XGBoost Fitting...')
    model_xgboost.fit(X_train_oh,y_train,eval_set=[(X_val_oh,y_val)],verbose=0)
    pred_xgboost  = model_xgboost.predict_proba(X_val_oh)
    p_xgboost = proba_multiclass_to_binary(pred_xgboost,power=proba_power)
    loss_xgboost = balanced_log_loss(np.where(y_val==0,0,1), p_xgboost[:, 1])
    
    # modeling - tabpfn
    pbar.set_description('[3/3] TabPFN Fitting...')
    model_tabpfn.fit(X_train_oh,y_train)
    pred_tabpfn = model_tabpfn.predict_proba(X_val_oh)
    p_tabpfn = proba_multiclass_to_binary(pred_tabpfn,power=proba_power)
    loss_tabpfn = balanced_log_loss(np.where(y_val==0,0,1), p_tabpfn[:, 1])
    
    # appending
    models += [model_catboost,model_xgboost,model_tabpfn]
    weights += [1/loss_catboost, 1/loss_xgboost, 1/loss_tabpfn]
    preds += [pred_catboost, pred_xgboost, pred_tabpfn]
    print([1/loss_catboost, 1/loss_xgboost, 1/loss_tabpfn])

[1/3] CatBoost Fitting...:  20%|██        | 1/5 [03:12<12:50, 192.64s/it]

[5.157170066838859, 3.21735287458906, 3.5877528682740087]


[1/3] CatBoost Fitting...:  40%|████      | 2/5 [08:01<12:27, 249.17s/it]

[10.522386720058682, 7.2081215820352496, 9.238794971231245]


[1/3] CatBoost Fitting...:  60%|██████    | 3/5 [12:13<08:21, 250.55s/it]

[5.4352062491735476, 2.906922647181294, 3.2647828974429447]


[1/3] CatBoost Fitting...:  80%|████████  | 4/5 [15:31<03:49, 229.79s/it]

[6.152115665418651, 5.089869427170583, 8.240076235512792]


[3/3] TabPFN Fitting...: 100%|██████████| 5/5 [22:10<00:00, 266.06s/it]

[8.166515086370426, 5.064131794275468, 4.623688666728858]
CPU times: user 1h 17min 12s, sys: 3min 30s, total: 1h 20min 43s
Wall time: 22min 10s


In [37]:
1/np.array(weights)

array([0.19390479, 0.31081452, 0.278726  , 0.09503547, 0.1387324 ,
       0.10823922, 0.18398566, 0.3440064 , 0.30629908, 0.16254571,
       0.19646869, 0.1213581 , 0.12245125, 0.19746721, 0.21627754])

<br></br>

# Inference

## Train

In [38]:
# 방법1 : get_weighted_proba
def get_predict_proba(model):
    if str(model).find('catboost')>=0:
        pred_proba = model.predict_proba(X_pool)
    else:
        pred_proba = model.predict_proba(X_oh)
    return pred_proba

X = train_df3[cat_features+num_features]
y = train_df3[target_feature]
X_oh = oh_encoder.transform(X)
X_pool = Pool(X,y,cat_features=cat_features)

# 5분
ps = [get_predict_proba(model) for model in tqdm(models)]
p = get_weighted_proba(ps)

100%|██████████| 15/15 [04:35<00:00, 18.38s/it]


In [39]:
print(balanced_log_loss(y_true=np.where(y==0,1,0),y_pred=p[:,1]))
pd.crosstab(np.where(y==0,1,0),np.where(p[:,1]>0.5,1,0))

1.707554299442405


col_0,0,1
row_0,,
0,0,108
1,486,23


In [40]:
%%time
# 방법2 : weighted sum
# 10분

X = train_df3[cat_features+num_features]
y = train_df3[target_feature]
X_oh = oh_encoder.transform(X)
X_pool = Pool(X,y,cat_features=cat_features)

normalized_weights = weights.copy()
normalized_weights /= sum(weights)

y_probas = np.zeros((len(X),2))
pbar = tqdm(zip(normalized_weights,models),total=len(models))
for weight,model in pbar:
    pbar.set_description(str(model).split('Classifier')[0])
    if str(model).find('catboost')>=0:
        pred_proba = model.predict_proba(X_pool)
    else:
        pred_proba = model.predict_proba(X_oh)
    pred_proba = get_weighted_proba([pred_proba])
    y_probas[:, 0] += weight * pred_proba[:, 0]
    y_probas[:, 1] += weight * pred_proba[:, 1]
y_probas /= sum(normalized_weights)

score = balanced_log_loss(y_true=np.where(y==0,0,1),y_pred=y_probas[:,1])
print('> Balanced Logloss: {:.4f}'.format(score))

TabPFN: 100%|██████████| 15/15 [04:34<00:00, 18.28s/it]

> Balanced Logloss: 0.0567
CPU times: user 7min 24s, sys: 1min 41s, total: 9min 5s
Wall time: 4min 34s


In [41]:
print(balanced_log_loss(y_true=np.where(y==0,0,1),y_pred=y_probas[:,1]))
pd.crosstab(np.where(y==0,0,1),np.where(y_probas[:,1]>0.5,1,0))

0.05670455055454736


col_0,0,1
row_0,,
0,499,10
1,0,108


In [42]:
# perpect_cutoff_list = np.arange(0.01,1.00,0.01)
# loss_list = []
# for perpect_cutoff in perpect_cutoff_list:
#     p = np.array([
#         1 if _p>perpect_cutoff else
#         0 if _p<1-perpect_cutoff else
#         _p for _p in y_probas[:,1]
#     ])
#     loss = balanced_log_loss(y_true=np.where(y==0,0,1),y_pred=p)
#     loss_list.append(loss)
    
# perpect_cutoff = perpect_cutoff_list[np.argmin(loss_list)]
# perpect_cutoff, min(loss_list)
# # (0.68, 0.015370262171626037)

In [43]:
perpect_cutoff = 0.9

p = np.array([
    1 if _p>perpect_cutoff else
    0 if _p<1-perpect_cutoff else
    _p for _p in y_probas[:,1]
])

In [44]:
print(balanced_log_loss(y_true=np.where(y==0,0,1),y_pred=p))
pd.crosstab(np.where(y==0,0,1),np.where(p>0.5,1,0))

0.04031020699254364


col_0,0,1
row_0,,
0,499,10
1,0,108


In [45]:
n_check = 40
y_probas[:,1][:n_check], p[:n_check]

(array([0.99004409, 0.04434641, 0.20876597, 0.28704236, 0.95464085,
        0.03399911, 0.16382117, 0.13211506, 0.08881177, 0.05879234,
        0.98235293, 0.11320526, 0.09225092, 0.98138482, 0.27878605,
        0.26565323, 0.12176372, 0.12295437, 0.24905388, 0.07909173,
        0.27998439, 0.07290713, 0.08549804, 0.05338045, 0.15696462,
        0.08419632, 0.26907518, 0.03400026, 0.06520853, 0.22892108,
        0.0763918 , 0.91206686, 0.98511162, 0.08163223, 0.14430544,
        0.03547691, 0.97875118, 0.03423529, 0.03593482, 0.12773998]),
 array([1.        , 0.        , 0.20876597, 0.28704236, 1.        ,
        0.        , 0.16382117, 0.13211506, 0.        , 0.        ,
        1.        , 0.11320526, 0.        , 1.        , 0.27878605,
        0.26565323, 0.12176372, 0.12295437, 0.24905388, 0.        ,
        0.27998439, 0.        , 0.        , 0.        , 0.15696462,
        0.        , 0.26907518, 0.        , 0.        , 0.22892108,
        0.        , 1.        , 1.        , 0.

<br>

## Test

In [46]:
%%time
# 10분

X_test = test_df3[cat_features+num_features]
X_test_oh = oh_encoder.transform(X_test)
X_test_pool = Pool(X_test,cat_features=cat_features)

normalized_weights = weights.copy()
normalized_weights /= sum(weights)

y_probas = np.zeros((len(X_test),2))
pbar = tqdm(zip(normalized_weights,models),total=len(models))
for weight,model in pbar:
    pbar.set_description(str(model).split('Classifier')[0])
    if str(model).find('catboost')>=0:
        pred_proba = model.predict_proba(X_test_pool)
    else:
        pred_proba = model.predict_proba(X_test_oh)
    y_probas[:, 0] += weight * pred_proba[:, 0]
    y_probas[:, 1] += weight * pred_proba[:, 1]
y_probas /= sum(normalized_weights)

TabPFN: 100%|██████████| 15/15 [02:12<00:00,  8.82s/it]

CPU times: user 3min 51s, sys: 30.8 s, total: 4min 22s
Wall time: 2min 12s


In [47]:
p = np.array([
    1 if _p>perpect_cutoff else
    0 if _p<1-perpect_cutoff else
    _p for _p in y_probas[:,1]
])

# p = y_probas[:,1]

In [48]:
y_probas[:,1], p

(array([0.10251965, 0.10251965, 0.10251965, 0.10251965, 0.10251965]),
 array([0.10251965, 0.10251965, 0.10251965, 0.10251965, 0.10251965]))

In [49]:
# submit = pd.read_csv('./data/sample_submission.csv')
submit = pd.read_csv(CFG.REF_PATH+'sample_submission.csv')
submit['class_0'] = p
submit['class_1'] = 1-p

In [50]:
submit

,Id,class_0,class_1
0,00eed32682bb,0.10252,0.89748
1,010ebe33f668,0.10252,0.89748
2,02fa521e1838,0.10252,0.89748
3,040e15f562a2,0.10252,0.89748
4,046e85c7cc7f,0.10252,0.89748


In [51]:
submit.to_csv('/kaggle/working/submission.csv', index=False)